# ERC-4626: list all vaults

This notebook serves both as a coding tutorial and a useful data analytics tool for ERC-4626 vaults. 

- In this notebook, we examine different ERC-4626 vaults across different EVM blockchains   
    - Currently we do not scan non-ERC-4626 vaults like Enzyme Finance, or any protocol-native vaults like Hyperliquid HPL. This is not an inherit limitation, this is not just yet implemented.
- We assemble various data tables out of the vault data to show and compare the blockchain ecosystems
- The analysis focus on USD-stablecoin nonminatd vaults
    - Currently missing are e.g. WETH vaults and staking vaults for various small cap tokens
    - There is no ERC standard for vaults fees - for some protocols we have manualled added fee reading support  
- The list of chains is somewhat randomly selected and very easy to extend to contain any chain supported by [Envio's HyperSync](https://docs.envio.dev/docs/HyperSync/hypersync-supported-networks)
- Everything is open source: You can run this notebook and associated scripts yourself on your local computer, it will take around an hour

In this notebook, we use terms Net Asset Value (NAV) and [Total Value Locked (TVL)](https://tradingstrategy.ai/glossary/total-value-locked-tvl) interchangeably.

## Usage

- Read general instructions [how to run the tutorials](./)
- See `ERC-4626 scanning all vaults onchain` example in tutorials first how to build a vault database as local `vault_db.pickle` file.




## Setup

- Set up notebook renderinb parmaeters

In [1]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

## Read scanned data

- Read the Pickle database our scanning script produced earlier 

In [2]:
import pickle
from pathlib import Path

import pandas as pd

from eth_defi.token import is_stablecoin_like

output_folder = Path("~/.tradingstrategy/vaults").expanduser()
vault_db = output_folder / "vault-db.pickle"
assert vault_db.exists(), "Run the vault scanner script first"

vault_db = pickle.load(open(vault_db, "rb"))

print(f"We have data for {len(vault_db)} vaults")

We have data for 8984 vaults


## Transform data

- Prepare the raw vault pickled data as Pandas DataFrame for data research

In [3]:
import datetime
from pprint import pformat
import pandas as pd
from eth_defi.erc_4626.hypersync_discovery import ERC4262VaultDetection
from eth_defi.chain import get_chain_name
from eth_defi.token import is_stablecoin_like

data = list(vault_db.values())
df = pd.DataFrame(data)

# print("Raw row example:")
# print(df.iloc[0])

# Build useful columns out of raw pickled Python data
# _detection_data contains entries as ERC4262VaultDetection class
entry: ERC4262VaultDetection
df["Chain"] = df["_detection_data"].apply(lambda entry: get_chain_name(entry.chain))
df["Protocol identified"] = df["_detection_data"].apply(lambda entry: entry.is_protocol_identifiable())
df["Stablecoin denominated"] = df["_denomination_token"].apply(lambda token_data: is_stablecoin_like(token_data["symbol"]) if pd.notna(token_data) else False)
df["ERC-7540"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7540())
df["ERC-7575"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7575())
df["Fee detected"] = df.apply(lambda row: (row["Mgmt fee"] is not None) or (row["Perf fee"] is not None), axis=1)
# Event counts
df["Deposit count"] = df["_detection_data"].apply(lambda entry: entry.deposit_count)
df["Redeem count"] = df["_detection_data"].apply(lambda entry: entry.redeem_count)
df["Total events"] = df["Deposit count"] + df["Redeem count"]
df["Mgmt fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Perf fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Age"] = datetime.datetime.utcnow() - df["First seen"]
df["NAV"] = df["NAV"].astype("float64")

df = df.sort_values(["Chain", "Address"])
df = df.set_index(["Chain", "Address"])

print("DataFrame MultiIndex is:", ", ".join(x for x in df.index.names))
print("DataFrame columns are:", ", ".join(x for x in df.columns))

source_df = df

display(df.sort_values("Total events", ascending=False).head())



DataFrame MultiIndex is: Chain, Address
DataFrame columns are: Symbol, Name, Denomination, NAV, Protocol, Mgmt fee, Perf fee, Shares, First seen, _detection_data, _denomination_token, _share_token, Protocol identified, Stablecoin denominated, ERC-7540, ERC-7575, Fee detected, Deposit count, Redeem count, Total events, Age


Symbol  \
Chain     Address                                                
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a   UB-WETH   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701      sDAI   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2    BBiBGT   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  yvUSDC-A   
          0xbb287e6017d3deb0e2e65061e8684eab21060123  yvUSDT-A   

                                                                 Name  \
Chain     Address                                                       
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a          UB-WETH   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701     Savings xDAI   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2  Beraborrow iBGT   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0    USDC yVault-A   
          0xbb287e6017d3deb0e2e65061e8684eab21060123    USDT yVault-A   

                                                     Denomination  \
Chain     Address                                                   
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a         WETH   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701        WXDAI   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2         iBGT   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0         USDC   
          0xbb287e6017d3deb0e2e65061e8684eab21060123         USDT   

                                                               NAV  \
Chain     Address                                                    
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a        428.69   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701 81,966,385.95   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2     87,582.51   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0    546,835.67   
          0xbb287e6017d3deb0e2e65061e8684eab21060123    410,671.03   

                                                                Protocol  \
Chain     Address                                                          
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a  <unknown ERC-4626>   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701  <unknown ERC-4626>   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2  <unknown ERC-4626>   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0            Yearn v3   
          0xbb287e6017d3deb0e2e65061e8684eab21060123            Yearn v3   

                                                       Mgmt fee   Perf fee  \
Chain     Address                                                            
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a  <unknown>  <unknown>   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701  <unknown>  <unknown>   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2  <unknown>  <unknown>   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  <unknown>  <unknown>   
          0xbb287e6017d3deb0e2e65061e8684eab21060123  <unknown>  <unknown>   

                                                                           Shares  \
Chain     Address                                                                   
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a       427.029869764692712211   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701  69605175.595232736040493392   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2     62450.219962448014365526   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0                462920.352252   
          0xbb287e6017d3deb0e2e65061e8684eab21060123                345250.232046   

                                                              First seen  \
Chain     Address                                                          
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a 2025-01-13 06:53:05   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701 2023-09-28 12:13:30   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2 2025-03-07 12:05:19   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2ea

## Vaults per chain summary

- Get a summary of scanned chains at what vaults they have
- *Generic* status means that we do not have classification rules to determine the protocol on which a particular ERC-4626 vault belongs
- *Broken* status means that we could not correctly extract ERC-4626 information out of a smart contract

To detect the protocol of a vault, we need to maintain a [manual rule list here](https://github.com/tradingstrategy-ai/web3-ethereum-defi/blob/master/eth_defi/erc_4626/classification.py). Not all protocols are supported at the moment. as there are too many protocols to manually examine and identify them. Open source contributions welcome.




In [4]:
nav_threshold = 100_000
broken_max_nav_threshold = 999_000_000_000

# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]

# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby(level='Chain').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby(level='Chain')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby(level='Chain').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby(level='Chain').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby(level='Chain').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby(level='Chain').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby(level='Chain').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby(level='Chain').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Chain,,,,,,,,
Arbitrum,1976,"179,545,463.03",354.00,687,58.00,22.00,30.00,224.00
Avalanche,320,"266,595,267.01",48.00,92,11.00,1.00,1.00,48.00
Base,1376,"463,587,061.75",640.00,525,64.00,39.00,18.00,118.00
Berachain,537,"73,695,197.54",49.00,30,17.00,0.00,0.00,5.00
Binance,363,"274,149,105.51",49.00,107,25.00,0.00,0.00,27.00
Blast,106,"1,831,927.63",2.00,14,2.00,0.00,0.00,32.00
Celo,9,"3,198,421.12",0.00,6,1.00,0.00,0.00,0.00
Ethereum,2242,"10,847,528,963.41",580.00,783,223.00,22.00,20.00,199.00
Gnosis,89,"84,253,604.49",13.00,37,4.00,0.00,0.00,0.00


## Vaults per protocol summary

- Break down by identified protocol
- Because of the brokeness of EVM, Solidity and smart contract development practices, protocol-rules are hand-maintained heurestics and there can false positives and negatives

In [5]:
df = source_df.copy()
df = df.reset_index()
# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]
# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby('Protocol').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby('Protocol')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby('Protocol').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby('Protocol').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby('Protocol').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby('Protocol').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby('Protocol').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby('Protocol').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
summary_df = summary_df.sort_values("Total vault TVL/NAV in stable vaults", ascending=False)
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Protocol,,,,,,,,
<unknown ERC-4626>,6448,"9,405,905,595.99",0.00,"2,239.00",259.00,0.00,12.00,0.00
Morpho,215,"1,485,444,160.97",215.00,124.00,69.00,0.00,0.00,215.00
Fluid,60,"485,338,495.78",60.00,26.00,8.00,0.00,0.00,0.00
<unknown ERC-7540>,48,"436,334,249.13",0.00,24.00,4.00,48.00,41.00,0.00
Euler Vault Kit,362,"402,028,808.12",362.00,99.00,39.00,0.00,0.00,0.00
Yearn v3,140,"88,527,269.71",140.00,70.00,20.00,0.00,0.00,0.00
Kiln Metavault,78,"51,155,862.97",78.00,67.00,25.00,0.00,0.00,0.00
Superform,142,"25,587,578.12",142.00,57.00,11.00,20.00,0.00,0.00
Gains Network,42,"11,063,040.64",42.00,18.00,10.00,3.00,0.00,0.00


## Vault deployment history

- Show how much history we have for each chain


In [6]:
# Assuming your DataFrame is named 'df'
df = source_df
seen_df = df.groupby(level='Chain')['First seen'].agg(['min', 'max']).reset_index()

# Rename columns for clarity
seen_df.columns = ['Chain', 'First vault deployed', 'Last vault deployed']

seen_df = seen_df.set_index("Chain")

display(seen_df)

,First vault deployed,Last vault deployed
Chain,,
Arbitrum,2022-03-28 09:03:11,2025-05-14 10:00:03
Avalanche,2022-04-11 00:05:24,2025-05-15 09:44:56
Base,2023-08-04 15:36:07,2025-05-16 05:56:57
Berachain,2025-01-26 00:06:03,2025-05-15 17:59:19
Binance,2022-05-27 17:25:18,2025-05-14 16:38:57
Blast,2024-02-29 20:15:37,2024-12-03 15:38:15
Celo,2022-08-08 21:14:10,2025-04-29 19:46:04
Ethereum,2019-06-11 06:17:19,2025-05-13 23:18:35
Gnosis,2022-05-01 18:47:35,2025-04-29 10:03:35


## Largest USD vaults

- Show the stablecoin-denominated vaults across different chains that have largest USD treasury 

In [7]:
largest_threshold = 20
largest_df = df.reset_index()
# Filter out crap
largest_df = largest_df[largest_df["Total events"] > 100] 
largest_df = largest_df[largest_df["Stablecoin denominated"] == True] 
largest_df = largest_df.sort_values(["NAV"], ascending=False)

largest_df = largest_df[["NAV", "Chain", "Address", "Name", "Denomination", "Total events", "Mgmt fee", "Perf fee"]]
largest_df = largest_df.set_index("Name")


display(largest_df.head(largest_threshold))


,NAV,Chain,Address,Denomination,Total events,Mgmt fee,Perf fee
Name,,,,,,,
Savings USDS,"2,335,418,583.13",Ethereum,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,34513,<unknown>,<unknown>
Staked USDe,"2,303,481,485.69",Ethereum,0x9d39a5de30e57443bff2a8307a4256c8797a3497,USDe,71612,<unknown>,<unknown>
Ethereal Pre-deposit Vault,"1,236,533,644.25",Ethereum,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,USDe,64213,<unknown>,<unknown>
Savings Dai,"564,246,487.81",Ethereum,0x83f20f44975d03b1b09e64809b757c47f942beea,DAI,74857,<unknown>,<unknown>
Syrup USDC,"336,428,112.21",Ethereum,0x80ac24aa929eaf5013f6436cda2a7ba190f5cc0b,USDC,5662,<unknown>,<unknown>
Spark DAI Vault,"275,061,667.08",Ethereum,0x73e65dbd630f90604062f6e02fab9138e713edd9,DAI,102,0.00,0.00
Spark USDC Vault,"258,135,211.27",Ethereum,0xbc65ad17c5c0a2a4d159fa5a503f4992c7b545fe,USDC,1530,<unknown>,<unknown>
Spark USDC Vault,"223,703,259.96",Base,0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a,USDC,15146,0.00,0.00
Staked Falcon USD,"222,460,313.06",Ethereum,0xc8cf6d7991f15525488b2a83df53468d682ba4b0,USDf,4406,<unknown>,<unknown>


## Largest USD vault per chain

- Get the largest vault of each chain

In [8]:
# Get the index of max NAV for each chain
largest_df = largest_df.reset_index().set_index(["Chain", "Name"])
max_nav_idx = largest_df.groupby('Chain')['NAV'].idxmax()
# Use these indices to get the full rows
result = largest_df.loc[max_nav_idx]

display(result)

NAV  \
Chain       Name                                                                  
Arbitrum    ostiumLP                                              66,021,525.80   
Avalanche   EVK Vault eUSDC-2                                     80,837,752.55   
Base        Spark USDC Vault                                     223,703,259.96   
            Spark USDC Vault                                      13,144,207.96   
Berachain   Concrete-USDe-Vault                                   16,602,180.54   
Binance     Staked USDX                                          203,588,378.95   
Blast       Wasabi USDB Vault                                      1,331,475.11   
Celo        Staked EURA                                            3,198,416.42   
Ethereum    Savings USDS                                       2,335,418,583.13   
Gnosis      Savings xDAI                                          81,966,385.95   
Hyperliquid hyUSDXL (Looped HYPE) - 1                              2,072,738.81   
Mode        USDC Ironclad Vault                                      337,647.27   
Optimism    exactly USDC                                           3,908,609.72   
Polygon     Compound USDC                                         24,253,779.31   
Soneium     UB-USDC                                                1,153,750.31   
Sonic       Silo Finance Borrowable USDC.e Deposit, SiloId: 20    63,297,220.56   
ZKsync      hsUSDC                                                   684,222.95   

                                                                                                   Address  \
Chain       Name                                                                                             
Arbitrum    ostiumLP                                            0x20d419a8e12c45f88fda7c5760bb6923cee27f98   
Avalanche   EVK Vault eUSDC-2                                   0x39de0f00189306062d79edec6dca5bb6bfd108f9   
Base        Spark USDC Vault                                    0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a   
            Spark USDC Vault                                    0x3128a0f7f0ea68e7b7c9b00afa7e41045828e858   
Berachain   Concrete-USDe-Vault                                 0x59e24f42cae1b82c8b2dc79ea898f2f8b4986dfc   
Binance     Staked USDX                                         0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92   
Blast       Wasabi USDB Vault                                   0x4bed2a922654cacc2be974689619768fabf24855   
Celo        Staked EURA                                         0x004626a008b1acdc4c74ab51644093b155e59a23   
Ethereum    Savings USDS                                        0xa3931d71877c0e7a3148cb7eb4463524fec27fbd   
Gnosis      Savings xDAI                                        0xaf204776c7245bf4147c2612bf6e5972ee483701   
Hyperliquid hyUSDXL (Looped HYPE) - 1                           0xaeedd5b6d42e0f077ccf3e7a78ff70b8cb217329   
Mode        USDC Ironclad Vault                                 0x882fd369341fc435ad5e54e91d1ebc23b1fc6d4c   
Optimism    exactly USDC                                        0x6926b434cce9b5b7966ae1bfeef6d0a7dcf3a8bb   
Polygon     Compound USDC                                       0x781fb7f6d845e3be129289833b04d43aa8558c42   
Soneium     UB-USDC                                             0xc675bb95d73ca7db2c09c3dc04daaa7944ccba41   
Sonic       Silo Finance Borrowable USDC.e Deposit, SiloId: 20  0x322e1d5384aa4ed66aeca770b95686271de61dc3   
ZKsync      hsUSDC                                              0xaf08a9d918f16332f22cf8dc9abe9d9e14ddcbc2   

                                                               Denomination  \
Chain       Name                                                              
Arbitrum    ostiumLP                                                   USDC   
Avalanche   EVK Vault eUSDC-2                                          USDC   
Base        Spark USDC Vault                                           USDC   
            

## Most active vaults across all chains

- Determine vault activity by number of deposit and redeem events
- Based on all-time event count, not recent event count 
- Events may be driven by bots, so this may not reflect the popularity of a vault amount users


In [9]:
largest_threshold = 20
largest_df = df.reset_index().sort_values(["Total events"], ascending=False)

largest_df = largest_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

largest_df = largest_df.set_index("Name")

display(largest_df.head(largest_threshold))

,Total events,Chain,Address,Denomination,NAV,Age,Deposit count,Redeem count
Name,,,,,,,,
UB-WETH,741249,Soneium,0x232554b4b291a446b4829300bec133fbb07a8f2a,WETH,428.69,123 days 11:37:41.205946,543105,198144
Savings xDAI,645963,Gnosis,0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"81,966,385.95",596 days 06:17:16.205946,350689,295274
Beraborrow iBGT,510656,Berachain,0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"87,582.51",70 days 06:25:27.205946,263493,247163
USDC yVault-A,490332,Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,USDC,"546,835.67",568 days 01:44:58.205946,465215,25117
USDT yVault-A,458396,Polygon,0xbb287e6017d3deb0e2e65061e8684eab21060123,USDT,"410,671.03",526 days 01:28:26.205946,407739,50657
BathToken v1,297341,Optimism,0xe0e112e8f33d3f437d1f895cbb1a456836125952,USDC,"60,366.75",1087 days 19:08:42.205946,150209,147132
Bridged USDC (Stargate)Vault,255306,Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,0.01,110 days 18:24:43.205946,114036,141270
UB-USDC,248668,Soneium,0xc675bb95d73ca7db2c09c3dc04daaa7944ccba41,USDC.e,"1,153,750.31",123 days 11:38:35.205946,176372,72296
Wasabi WETH Vault,243212,Blast,0x8e2b50413a53f50e2a059142a9be060294961e40,WETH,793.26,441 days 22:15:09.205946,161268,81944


## Most historically active vault per chain

- Vaults with most deposit and redeem events

In [10]:
most_active_df = df.reset_index()

most_active_df = most_active_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

# Force thousand separator
most_active_df["Total events"] = most_active_df["Total events"].astype("float64")

max_nav_idx = most_active_df.groupby('Chain')['Total events'].idxmax()
# Use these indices to get the full rows
result = most_active_df.loc[max_nav_idx]

result = result.set_index(["Chain", "Name"])

display(result)

,,Total events,Address,Denomination,NAV,Age,Deposit count,Redeem count
Chain,Name,,,,,,,
Arbitrum,Staked USDA,"122,507.00",0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"938,444.31",491 days 09:58:21.205946,120201,2306
Avalanche,GoGoPool Liquid Staking Token,"82,710.00",0xa25eaf2906fa1a3a13edac9b9657108af7b703e3,WAVAX,"887,057.54",771 days 21:20:01.205946,48587,34123
Base,Moonwell Flagship ETH,"149,560.00",0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"12,233.64",338 days 05:33:31.205946,89423,60137
Berachain,Beraborrow iBGT,"510,656.00",0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"87,582.51",70 days 06:25:27.205946,263493,247163
Binance,kUSDT,"43,081.00",0x1c3f35f7883fc4ea8c4bca1507144dc6087ad0fb,VUSD,"1,467,100.85",704 days 11:22:03.205946,25386,17695
Blast,Wasabi WETH Vault,"243,212.00",0x8e2b50413a53f50e2a059142a9be060294961e40,WETH,793.26,441 days 22:15:09.205946,161268,81944
Celo,Wrapped mcUSD,241.00,0x5f71fe197fd9aad95b7b5510a98882769622112a,mcUSD,"32,075.35",973 days 22:56:49.205946,180,61
Ethereum,pufETH,"168,806.00",0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"68,742.72",470 days 22:46:11.205946,152103,16703
Gnosis,Savings xDAI,"645,963.00",0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"81,966,385.95",596 days 06:17:16.205946,350689,295274


## Oldest vaults

- Show oldest vaults

In [11]:
threshold = 1_000

oldest_df = df.reset_index()

oldest_df = oldest_df[["Chain", "Address", "Name", "Age", "Denomination", "NAV", "Total events"]]

# Force thousand separator
oldest_df["Total events"] = oldest_df["Total events"].astype("float64")

# Force event threshold to filter out some crap
oldest_df = oldest_df[oldest_df["Total events"] >= threshold]

max_nav_idx = oldest_df.groupby('Chain')['Age'].idxmax()
# Use these indices to get the full rows
result = oldest_df.loc[max_nav_idx]

result = result.set_index("Chain")

display(result)

,Address,Name,Age,Denomination,NAV,Total events
Chain,,,,,,
Arbitrum,0xf46ce0c13577232d5f29d9bd78a9cab278755346,Jones ETH,1085 days 03:15:40.205946,WETH,17.99,"2,338.00"
Avalanche,0x9dd17f32fc8355ae37425f475a10cc7bec8ca36a,,1129 days 01:33:24.205946,,0.00,"1,425.00"
Base,0xc7548d8d7560f6679e369d0556c44fe1eddea3e9,FARM_WETH,624 days 02:36:53.205946,WETH,0.97,"1,119.00"
Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,Bridged USDC (Stargate)Vault,110 days 18:24:43.205946,USDC.e,0.01,"255,306.00"
Binance,0x0f8754b36a767c5579178bd8a04d2fcd9d530b70,ygNRCH,1053 days 21:38:07.205946,NRCH,"1,175,635.63","1,198.00"
Blast,0x8e2b50413a53f50e2a059142a9be060294961e40,Wasabi WETH Vault,441 days 22:15:09.205946,WETH,793.26,"243,212.00"
Ethereum,0x815c23eca83261b6ec689b60cc4a58b54bc24d8d,vTHOR,1117 days 07:05:09.205946,THOR,"88,639,913.00","19,985.00"
Gnosis,0x004626a008b1acdc4c74ab51644093b155e59a23,Staked EURA,623 days 04:48:51.205946,EURA,"525,767.44","3,639.00"
Hyperliquid,0x2831775cb5e64b1d892853893858a261e898fbeb,wHYPE,64 days 20:19:46.205946,WHYPE,"395,141.21","2,671.00"


## All vaults

- List of all stablecoin vaults

In [12]:
all_df = df.reset_index()

min_nav = 10_000

stablecoin_denominated = all_df["Stablecoin denominated"] == True
all_df = all_df[stablecoin_denominated]
all_df = all_df[all_df["NAV"] < broken_max_nav_threshold]
all_df = all_df[all_df["NAV"] >= min_nav]
all_df = all_df[["NAV", "Chain", "Protocol", "Address", "Name", "Age", "Denomination", "Total events"]]
all_df = all_df.set_index("Name").sort_values("NAV", ascending=False)

print(f"Displaying {len(all_df)} stablecoin vaults w/min NAV of {min_nav} USD")
pd.options.display.max_rows = None
display(all_df)


Displaying 673 stablecoin vaults w/min NAV of 10000 USD


,NAV,Chain,Protocol,Address,Age,Denomination,Total events
Name,,,,,,,
Savings USDS,"2,335,418,583.13",Ethereum,<unknown ERC-4626>,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,241 days 04:53:59.205946,USDS,34513
Staked USDe,"2,303,481,485.69",Ethereum,<unknown ERC-4626>,0x9d39a5de30e57443bff2a8307a4256c8797a3497,542 days 10:24:35.205946,USDe,71612
Ethereal Pre-deposit Vault,"1,236,533,644.25",Ethereum,<unknown ERC-4626>,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,92 days 03:16:35.205946,USDe,64213
Savings Dai,"564,246,487.81",Ethereum,<unknown ERC-4626>,0x83f20f44975d03b1b09e64809b757c47f942beea,779 days 08:05:59.205946,DAI,74857
Janus Henderson Anemoy Treasury Fund,"429,933,704.21",Ethereum,<unknown ERC-7540>,0x36036ffd9b1c6966ab23209e073c68eb9a992f50,38 days 21:18:59.205946,USDC,7
Syrup USDC,"336,428,112.21",Ethereum,<unknown ERC-4626>,0x80ac24aa929eaf5013f6436cda2a7ba190f5cc0b,334 days 23:59:35.205946,USDC,5662
Spark DAI Vault,"275,061,667.08",Ethereum,Morpho,0x73e65dbd630f90604062f6e02fab9138e713edd9,413 days 04:10:59.205946,DAI,102
Spark USDC Vault,"258,135,211.27",Ethereum,<unknown ERC-4626>,0xbc65ad17c5c0a2a4d159fa5a503f4992c7b545fe,72 days 03:30:35.205946,USDC,1530
Spark USDC Vault,"223,703,259.96",Base,Morpho,0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a,120 days 21:59:31.205946,USDC,15146
